In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

In [2]:
detailed_season_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv")
detailed_season_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


### Calculation of Advanced Metrics per Game

In [3]:
advanved_detailed_season_df = detailed_season_df.copy()

# Drop DayNum and NumOT as irrelevant
advanved_detailed_season_df = advanved_detailed_season_df.drop(columns = ["DayNum", "NumOT"])

# Effective Field Goal Percentage
# Winning Team: WFGM + (0.5 * WFGM3) / WFGA
advanved_detailed_season_df['WEFGP'] = (advanved_detailed_season_df['WFGM'] + (0.5 * advanved_detailed_season_df['WFGM3'])) / advanved_detailed_season_df['WFGA']
# Losing Team: LFGM + (0.5 * LFGM3) / LFGA
advanved_detailed_season_df['LEFGP'] = (advanved_detailed_season_df['LFGM'] + (0.5 * advanved_detailed_season_df['LFGM3'])) / advanved_detailed_season_df['LFGA']

#Turnover Percentage
# Winning Team: WTO / (WFGA + (0.44 * WFTA) + WTO)
advanved_detailed_season_df['WTP'] = advanved_detailed_season_df['WTO'] / (advanved_detailed_season_df['WFGA'] + (0.44 * advanved_detailed_season_df['WFTA']) + advanved_detailed_season_df['WTO']) 
# Losing Team: LTO / (LFGA + (0.44 * LFTA) + LTO)
advanved_detailed_season_df['LTP'] = advanved_detailed_season_df['LTO'] / (advanved_detailed_season_df['LFGA'] + (0.44 * advanved_detailed_season_df['LFTA']) + advanved_detailed_season_df['LTO']) 

# Offensive Rebound Percentage
# Winning Team: WOR / WOR + LDR
advanved_detailed_season_df['WORP'] = advanved_detailed_season_df['WOR'] / (advanved_detailed_season_df['WOR'] + advanved_detailed_season_df['LDR'])
#Losing Team: LOR / LOR + WDR
advanved_detailed_season_df['LORP'] = advanved_detailed_season_df['LOR'] / (advanved_detailed_season_df['LOR'] + advanved_detailed_season_df['WDR'])

# Free Throw Rate
# Winning Team: WFTM / WFGA
advanved_detailed_season_df['WFTR'] = advanved_detailed_season_df['WFTM'] / advanved_detailed_season_df['WFGA'] 
# Losing Team: LFTM / LFGA
advanved_detailed_season_df['LFTR'] = advanved_detailed_season_df['LFTM'] / advanved_detailed_season_df['LFGA'] 

advanved_detailed_season_df.head()

,Season,WTeamID,WScore,LTeamID,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,...,LBlk,LPF,WEFGP,LEFGP,WTP,LTP,WORP,LORP,WFTR,LFTR
0,2003,1104,68,1328,62,N,27,58,3,14,...,2,20,0.491379,0.433962,0.258659,0.223104,0.388889,0.294118,0.189655,0.301887
1,2003,1272,70,1393,63,N,26,62,8,20,...,6,16,0.483871,0.402985,0.155950,0.136674,0.375000,0.416667,0.161290,0.134328
2,2003,1266,73,1437,61,N,24,58,8,18,...,5,23,0.482759,0.321918,0.123824,0.126156,0.435897,0.543860,0.293103,0.191781
3,2003,1296,56,1457,50,N,18,38,3,9,...,3,23,0.513158,0.428571,0.188561,0.254692,0.230769,0.472222,0.447368,0.163265
4,2003,1400,77,1208,71,N,30,61,6,14,...,1,14,0.540984,0.435484,0.173439,0.119218,0.531250,0.488372,0.180328,0.274194


### Calculation of Advanced Metrics by Season

In [4]:
# Determining which columns pertain to winners and losers
W_cols = []
L_cols = []

for c in advanved_detailed_season_df.columns:
    if c[0] == "W": #Column pertains to winner
        W_cols.append(c)
    elif c[0] == "L": # Column pertains to loser
        L_cols.append(c)
    else: # Column applies to both
        W_cols.append(c)
        L_cols.append(c)

In [5]:
winners_season_totals_df = advanved_detailed_season_df[W_cols].groupby(["Season","WTeamID"]).sum().reset_index()
winners_season_totals_df.head()


,Season,WTeamID,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,WEFGP,WTP,WORP,WFTR
0,2003,1102,825,271,480,120,259,163,249,46,232,203,133,88,34,193,8.305950,2.184427,2.066250,4.270526
1,2003,1103,1141,390,720,71,187,290,402,122,279,230,163,95,24,266,7.703849,2.017537,4.140626,5.224313
2,2003,1104,1270,439,992,120,354,272,383,230,449,238,222,123,71,280,8.602946,2.724733,6.239836,4.728497
3,2003,1105,556,179,433,64,157,134,180,102,181,111,126,79,14,136,3.418803,1.382465,2.682010,2.202557
4,2003,1106,888,322,700,76,207,168,270,166,364,169,230,122,49,239,6.729689,2.862692,4.926373,3.225302


In [6]:
losers_season_totals_df = advanved_detailed_season_df[L_cols].groupby(["Season","LTeamID"]).sum().reset_index()
losers_season_totals_df.head()


,Season,LTeamID,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,LEFGP,LTP,LORP,LFTR
0,2003,1102,778,265,634,99,324,149,230,71,239,161,187,79,16,332,8.057458,3.222974,2.644341,3.889684
1,2003,1103,986,343,788,76,247,224,296,142,259,181,178,101,39,270,6.783384,2.297189,4.116043,4.036028
2,2003,1104,670,234,609,58,202,144,203,150,221,101,150,62,35,225,4.719024,1.958960,4.155320,2.659445
3,2003,1105,1310,455,1169,133,383,267,388,249,420,267,359,163,40,390,8.488761,4.020133,6.032299,4.443809
4,2003,1106,893,334,848,95,287,130,191,178,304,158,247,112,39,270,6.757827,3.121495,4.859077,2.367428


### Calculating Num Wins & Losses per Team by Year

In [7]:
# Read in data
compact_season_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv")
# Filter to Season > 2003 (min year for deatailed daya)
compact_season_df = compact_season_df[compact_season_df["Season"] >= 2003].reset_index(drop=True)
compact_season_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,10,1104,68,1328,62,N,0
1,2003,10,1272,70,1393,63,N,0
2,2003,11,1266,73,1437,61,N,0
3,2003,11,1296,56,1457,50,N,0
4,2003,11,1400,77,1208,71,N,0


In [8]:
# Group by Season and WTeamID, count rows, replace NA's w/ 0's for undefeated teams
wins_per_season_df = compact_season_df.groupby(["Season","WTeamID"])["DayNum"].count().reset_index().fillna(0)
wins_per_season_df.rename(columns = {"DayNum": "NumWins"}, inplace = True)

wins_per_season_df.head()

,Season,WTeamID,NumWins
0,2003,1102,12
1,2003,1103,13
2,2003,1104,17
3,2003,1105,7
4,2003,1106,13


In [9]:
# Group by Season and LTeamID, count rows, replace NA's w/ 0's for teams without win
losses_per_season_df = compact_season_df.groupby(["Season","LTeamID"])["DayNum"].count().reset_index().fillna(0)
losses_per_season_df.rename(columns = {"DayNum": "NumLosses"}, inplace = True)

losses_per_season_df.head()

,Season,LTeamID,NumLosses
0,2003,1102,16
1,2003,1103,14
2,2003,1104,11
3,2003,1105,19
4,2003,1106,15


### Computing Weighted avg. Advanced Metrics per Team by Season

In [10]:
combined_winners_df = wins_per_season_df.merge(winners_season_totals_df, on=["Season","WTeamID"])
combined_winners_df.head()

,Season,WTeamID,NumWins,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,...,WDR,WAst,WTO,WStl,WBlk,WPF,WEFGP,WTP,WORP,WFTR
0,2003,1102,12,825,271,480,120,259,163,249,...,232,203,133,88,34,193,8.305950,2.184427,2.066250,4.270526
1,2003,1103,13,1141,390,720,71,187,290,402,...,279,230,163,95,24,266,7.703849,2.017537,4.140626,5.224313
2,2003,1104,17,1270,439,992,120,354,272,383,...,449,238,222,123,71,280,8.602946,2.724733,6.239836,4.728497
3,2003,1105,7,556,179,433,64,157,134,180,...,181,111,126,79,14,136,3.418803,1.382465,2.682010,2.202557
4,2003,1106,13,888,322,700,76,207,168,270,...,364,169,230,122,49,239,6.729689,2.862692,4.926373,3.225302


In [11]:
combined_losers_df = losses_per_season_df.merge(losers_season_totals_df, on=["Season","LTeamID"])
combined_losers_df.head()

,Season,LTeamID,NumLosses,LScore,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,...,LDR,LAst,LTO,LStl,LBlk,LPF,LEFGP,LTP,LORP,LFTR
0,2003,1102,16,778,265,634,99,324,149,230,...,239,161,187,79,16,332,8.057458,3.222974,2.644341,3.889684
1,2003,1103,14,986,343,788,76,247,224,296,...,259,181,178,101,39,270,6.783384,2.297189,4.116043,4.036028
2,2003,1104,11,670,234,609,58,202,144,203,...,221,101,150,62,35,225,4.719024,1.958960,4.155320,2.659445
3,2003,1105,19,1310,455,1169,133,383,267,388,...,420,267,359,163,40,390,8.488761,4.020133,6.032299,4.443809
4,2003,1106,15,893,334,848,95,287,130,191,...,304,158,247,112,39,270,6.757827,3.121495,4.859077,2.367428


In [12]:
combined_season_df = combined_winners_df.merge(combined_losers_df, left_on=["Season", "WTeamID"], right_on=["Season", "LTeamID"])
combined_season_df.head()


,Season,WTeamID,NumWins,WScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,...,LDR,LAst,LTO,LStl,LBlk,LPF,LEFGP,LTP,LORP,LFTR
0,2003,1102,12,825,271,480,120,259,163,249,...,239,161,187,79,16,332,8.057458,3.222974,2.644341,3.889684
1,2003,1103,13,1141,390,720,71,187,290,402,...,259,181,178,101,39,270,6.783384,2.297189,4.116043,4.036028
2,2003,1104,17,1270,439,992,120,354,272,383,...,221,101,150,62,35,225,4.719024,1.958960,4.155320,2.659445
3,2003,1105,7,556,179,433,64,157,134,180,...,420,267,359,163,40,390,8.488761,4.020133,6.032299,4.443809
4,2003,1106,13,888,322,700,76,207,168,270,...,304,158,247,112,39,270,6.757827,3.121495,4.859077,2.367428


In [13]:
# Make sure both DataFrames have same number of columns
assert(len(combined_winners_df.columns) == len(combined_losers_df.columns))

# Create new DataFrame to store weighted results
weighted_season_df = pd.DataFrame()

# Iterate through features, 
for idx in range(len(combined_winners_df.columns)): 
    
    win_col_name = combined_winners_df.columns[idx]
    
    # Set new columnn name removing W/L prefix
    if win_col_name[0] == "W":
        new_col_name = "avg_" + win_col_name[1:]
    else:
        new_col_name =  win_col_name
        
    # Take weighted avg of stats from wins and losses
    weighted_season_df[new_col_name] = (combined_season_df[combined_winners_df.columns[idx]] + combined_season_df[combined_losers_df.columns[idx]]) / (combined_season_df["NumWins"] + combined_season_df["NumLosses"]) 


weighted_season_df.rename(columns = {"avg_TeamID": "TeamID"}, inplace = True)
weighted_season_df["Season"] = combined_season_df["Season"]
weighted_season_df["TeamID"] = combined_season_df["WTeamID"]
weighted_season_df["NumLosses"] = combined_season_df["NumLosses"]
weighted_season_df["NumWins"] = combined_season_df["NumWins"]
weighted_season_df["NumLosses"] = combined_season_df["NumLosses"]

weighted_season_df.head()


,Season,TeamID,NumWins,avg_Score,avg_FGM,avg_FGA,avg_FGM3,avg_FGA3,avg_FTM,avg_FTA,...,avg_Ast,avg_TO,avg_Stl,avg_Blk,avg_PF,avg_EFGP,avg_TP,avg_ORP,avg_FTR,NumLosses
0,2003,1102,12,57.250000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,...,13.000000,11.428571,5.964286,1.785714,18.750000,0.584407,0.193121,0.168235,0.291436,16
1,2003,1103,13,78.777778,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,...,15.222222,12.629630,7.259259,2.333333,19.851852,0.536564,0.159805,0.305803,0.342976,14
2,2003,1104,17,69.285714,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,...,12.107143,13.285714,6.607143,3.785714,18.035714,0.475785,0.167275,0.371256,0.263855,11
3,2003,1105,7,71.769231,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,...,14.538462,18.653846,9.307692,2.076923,20.230769,0.457983,0.207792,0.335166,0.255629,19
4,2003,1106,13,63.607143,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,...,11.678571,17.035714,8.357143,3.142857,18.178571,0.481697,0.213721,0.349480,0.199740,15


# Combo

In [14]:
compact_tournament_df = pd.read_csv("../data//Kaggle-Data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv")
compact_tournament_df = compact_tournament_df[compact_tournament_df["Season"] >= 2003].reset_index(drop=True)
compact_tournament_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,134,1421,92,1411,84,N,1
1,2003,136,1112,80,1436,51,N,0
2,2003,136,1113,84,1272,71,N,0
3,2003,136,1141,79,1166,73,N,0
4,2003,136,1143,76,1301,74,N,1
...,...,...,...,...,...,...,...,...
1176,2021,148,1211,85,1425,66,N,0
1177,2021,148,1417,51,1276,49,N,0
1178,2021,152,1124,78,1222,59,N,0
1179,2021,152,1211,93,1417,90,N,1


In [15]:
winners = compact_tournament_df[["Season","WTeamID","LTeamID"]].merge(weighted_season_df, how="left", 
                                                            left_on=["Season","WTeamID"], 
                                                            right_on = ["Season","TeamID"])

losers = compact_tournament_df[["Season","WTeamID","LTeamID"]].merge(weighted_season_df, how="left", 
                                                            left_on=["Season","LTeamID"], 
                                                            right_on = ["Season","TeamID"])

In [76]:
num_games = len(compact_tournament_df.index)
random_selection = np.random.choice(a=[0,1], size=num_games)

ATeamID = np.empty(num_games)
BTeamID = np.empty(num_games)
AScore = np.empty(num_games)
BScore = np.empty(num_games)
Winner_label = np.empty(num_games)

for game_num in range(num_games):
    # Randomly choose number
    random_scramble = np.random.choice([0,1])
    # Number = 1: Team A won, Number = 0: Team B won
    Winner_label[game_num] = random_scramble
    
    if random_scramble == 1:
        ATeamID[game_num] = compact_tournament_df['WTeamID'][game_num]
        AScore[game_num] = compact_tournament_df['WScore'][game_num]
        
        BTeamID[game_num] = compact_tournament_df['LTeamID'][game_num]
        BScore[game_num] = compact_tournament_df['LScore'][game_num]
    else:
        ATeamID[game_num] = compact_tournament_df['LTeamID'][game_num]
        AScore[game_num] = compact_tournament_df['LScore'][game_num]
        
        BTeamID[game_num] = compact_tournament_df['WTeamID'][game_num]
        BScore[game_num] = compact_tournament_df['WScore'][game_num]

In [77]:
scrambled_tournament_df = pd.DataFrame({"Season":compact_tournament_df['Season'],
                                        "DayNum":compact_tournament_df['DayNum'],
                                        "WinnerLabel":Winner_label,
                                        "ATeamID":ATeamID, 
                                        "AScore":AScore, 
                                        "BTeamID":BTeamID, 
                                        "BScore":BScore,
                                        "NumOT": compact_tournament_df['NumOT']})

scrambled_tournament_df.head()
                                                                                                                                                                        

,Season,DayNum,WinnerLabel,ATeamID,AScore,BTeamID,BScore,NumOT
0,2003,134,0.0,1411.0,84.0,1421.0,92.0,1
1,2003,136,1.0,1112.0,80.0,1436.0,51.0,0
2,2003,136,0.0,1272.0,71.0,1113.0,84.0,0
3,2003,136,0.0,1166.0,73.0,1141.0,79.0,0
4,2003,136,0.0,1301.0,74.0,1143.0,76.0,1


In [127]:
A_stats = weighted_season_df.add_prefix('A_')
B_stats = weighted_season_df.add_prefix('B_')

half_joined_df = scrambled_tournament_df.merge(A_stats, how="left", 
                              left_on=["Season","ATeamID"], 
                              right_on = ["A_Season","A_TeamID"])

full_joined_df = half_joined_df.merge(B_stats, how="left", 
                              left_on=["Season","BTeamID"], 
                              right_on = ["B_Season","B_TeamID"])

full_joined_df = full_joined_df.dropna()
full_joined_df = full_joined_df.reset_index(drop=True)

full_joined_df.head()

,Season,DayNum,WinnerLabel,ATeamID,AScore,BTeamID,BScore,NumOT,A_Season,A_TeamID,...,B_avg_Ast,B_avg_TO,B_avg_Stl,B_avg_Blk,B_avg_PF,B_avg_EFGP,B_avg_TP,B_avg_ORP,B_avg_FTR,B_NumLosses
0,2003,134,0.0,1411.0,84.0,1421.0,92.0,1,2003.0,1411.0,...,13.034483,16.206897,7.068966,3.000000,19.103448,0.489800,0.196965,0.347184,0.287716,16.0
1,2003,136,1.0,1112.0,80.0,1436.0,51.0,0,2003.0,1112.0,...,14.206897,14.068966,6.862069,2.965517,15.896552,0.494732,0.180141,0.380016,0.235084,10.0
2,2003,136,0.0,1272.0,71.0,1113.0,84.0,0,2003.0,1272.0,...,15.551724,14.000000,5.206897,4.241379,19.413793,0.517334,0.170036,0.398080,0.313270,11.0
3,2003,136,0.0,1166.0,73.0,1141.0,79.0,0,2003.0,1166.0,...,15.620690,18.241379,7.103448,4.000000,20.965517,0.572835,0.223575,0.359177,0.369588,6.0
4,2003,136,0.0,1301.0,74.0,1143.0,76.0,1,2003.0,1301.0,...,16.000000,14.172414,6.551724,2.793103,17.103448,0.524098,0.173660,0.322310,0.240967,8.0


In [133]:
feature_columns = ['A_NumWins', 'A_avg_Score', 'A_avg_FGM',
       'A_avg_FGA', 'A_avg_FGM3', 'A_avg_FGA3', 'A_avg_FTM', 'A_avg_FTA',
       'A_avg_OR', 'A_avg_DR', 'A_avg_Ast', 'A_avg_TO', 'A_avg_Stl',
       'A_avg_Blk', 'A_avg_PF', 'A_avg_EFGP', 'A_avg_TP', 'A_avg_ORP',
       'A_avg_FTR', 'A_NumLosses', 'B_NumWins',
       'B_avg_Score', 'B_avg_FGM', 'B_avg_FGA', 'B_avg_FGM3', 'B_avg_FGA3',
       'B_avg_FTM', 'B_avg_FTA', 'B_avg_OR', 'B_avg_DR', 'B_avg_Ast',
       'B_avg_TO', 'B_avg_Stl', 'B_avg_Blk', 'B_avg_PF', 'B_avg_EFGP',
       'B_avg_TP', 'B_avg_ORP', 'B_avg_FTR', 'B_NumLosses']

feature_df = full_joined_df[feature_columns]

feature_df.head()

,A_NumWins,A_avg_Score,A_avg_FGM,A_avg_FGA,A_avg_FGM3,A_avg_FGA3,A_avg_FTM,A_avg_FTA,A_avg_OR,A_avg_DR,...,B_avg_Ast,B_avg_TO,B_avg_Stl,B_avg_Blk,B_avg_PF,B_avg_EFGP,B_avg_TP,B_avg_ORP,B_avg_FTR,B_NumLosses
0,18.0,72.800000,24.733333,55.266667,5.933333,18.500000,17.400000,28.066667,13.166667,24.800000,...,13.034483,16.206897,7.068966,3.000000,19.103448,0.489800,0.196965,0.347184,0.287716,16.0
1,25.0,85.214286,30.321429,65.714286,7.035714,20.071429,17.535714,25.000000,15.178571,27.642857,...,14.206897,14.068966,6.862069,2.965517,15.896552,0.494732,0.180141,0.380016,0.235084,10.0
2,23.0,74.517241,26.275862,60.000000,7.000000,20.068966,14.965517,22.896552,14.068966,25.965517,...,15.551724,14.000000,5.206897,4.241379,19.413793,0.517334,0.170036,0.398080,0.313270,11.0
3,29.0,79.242424,28.696970,57.454545,7.969697,20.484848,13.878788,20.030303,10.878788,23.181818,...,15.620690,18.241379,7.103448,4.000000,20.965517,0.572835,0.223575,0.359177,0.369588,6.0
4,18.0,72.400000,24.333333,53.333333,7.966667,22.500000,15.766667,20.466667,9.733333,22.033333,...,16.000000,14.172414,6.551724,2.793103,17.103448,0.524098,0.173660,0.322310,0.240967,8.0


# LR Model

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression




In [135]:
x_train, x_test, y_train, y_test = train_test_split(feature_df, full_joined_df['WinnerLabel'], test_size=0.1)
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
score = logisticRegr.score(x_test, y_test)
print(score)

0.7008547008547008


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [117]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [149]:
svm = SVC()
svm.fit(x_train, y_train)

y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.68      0.77      0.72        66
         1.0       0.64      0.53      0.58        51

    accuracy                           0.67       117
   macro avg       0.66      0.65      0.65       117
weighted avg       0.66      0.67      0.66       117



## Betting

In [109]:
moneyline_df = pd.read_csv("../data/Pre-Processed-Data/CleanedMoneylineData.csv",index_col=0)
moneyline_df.head()

,Season,DayNum,TeamID,Moneyline
0,2008,0,1263,200.0
1,2008,0,1350,-240.0
2,2008,0,1404,13000.0
3,2008,0,1272,-39000.0
4,2008,1,1205,-160.0


In [110]:
def calc_net_payout(moneyline, amount_bet = 1):
    if moneyline < 0:
        return -amount_bet*(100/moneyline)
    else:
        return amount_bet*(moneyline/100)

In [157]:
test_bet_output = full_joined_df.iloc[y_test.index][['Season', 'DayNum', 'ATeamID', 'BTeamID','WinnerLabel']]
test_bet_output['Prediction'] = y_pred
test_bet_output.head()

,Season,DayNum,ATeamID,BTeamID,WinnerLabel,Prediction
497,2010,143,1393.0,1139.0,0.0,1.0
1112,2021,137,1159.0,1116.0,0.0,0.0
954,2017,139,1257.0,1276.0,0.0,1.0
823,2015,139,1277.0,1438.0,1.0,0.0
967,2017,145,1211.0,1462.0,1.0,1.0


In [168]:
val = moneyline_df[(moneyline_df["Season"] == 2010) & 
                   (moneyline_df["DayNum"] == 143) & (moneyline_df["TeamID"] == 1393)]["Moneyline"].item()

In [ ]:
def get_moneyline(season, daynum, teamID):
    moneyline_df

In [170]:
initial_investment = len(test_bet_output.index)
running_balance = initial_investment

for game_num in test_bet_output.index:
    
    prediction = test_bet_output['Prediction'][game_num]
    label = test_bet_output['WinnerLabel'][game_num]
    season = test_bet_output["Season"]
    daynum = test_bet_output["DayNum"]
    
    if prediction == 1: # We Bet on A
        teamID = test_bet_output["ATeamID"]
    else: # We bet on B
        teamID = test_bet_output["BTeamID"]
        
    
    if prediction == label: # We were correct
        if label == 1: # Team A won
            
            moneyline = moneyline_df[(moneyline_df["Season"] == season) &
                                     (moneyline_df["DayNum"] == daynum) &
                                     (moneyline_df["TeamID"] == teamID)]["Moneyline"].item()
            print("won bet on A")
        else:
            print("won bet on B")
    
    else: # We were wrong
        if prediction == 1: # We bet on A
            print("lost bet on A")
        else: 
            print("lost bet on B")
            

lost bet on A
won bet on B
lost bet on A
lost bet on B


ValueError: Can only compare identically-labeled Series objects

In [138]:
full_joined_df.index

RangeIndex(start=0, stop=1168, step=1)